## Paper recycling
A paper recycling company converts newspaper, mixed paper, white office paper, and cardboard into pulp for newsprint, packaging paper, and print stock quality paper. The following table summarizes the yield for each kind of pulp recovered from each ton of recycled material and the cost of processing each ton of raw material into the various types of pulp.

**Step 1: Set up and import Gurobi**

In [ ]:
# Install Gurobi
!pip install gurobipy
from typing import List

# Import the Gurobi Package
import gurobipy as gp
from gurobipy import GRB

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 84.3 MB/s eta 0:00:00


**Step 2: Data Entry**

In [ ]:
# Specify input data
materials, purchase_cost, availability = gp.multidict ({
    'Newspaper'          : [15, 600],
    'Mixed paper'        : [16, 500],
    'White office paper' : [19, 300],
    'Cardboard'          : [17, 400]})

products, product_demand = gp.multidict ({
    'Newsprint'   : 500,
    'Packaging'   : 600,
    'Print stock' : 300})

material_product_pairs, recycling_yield, processing_cost = gp.multidict ({
    ('Newspaper', 'Newsprint')            : [0.85,  6.50],
    ('Newspaper', 'Packaging')            : [0.80, 11.00],
    ('Mixed paper', 'Newsprint')          : [0.90,  9.75],
    ('Mixed paper', 'Packaging')          : [0.80, 12.25],
    ('Mixed paper', 'Print stock')        : [0.70,  9.50],
    ('White office paper', 'Newsprint')   : [0.90,  4.75],
    ('White office paper', 'Packaging')   : [0.85,  7.75],
    ('White office paper', 'Print stock') : [0.80,  8.50],
    ('Cardboard', 'Newsprint')            : [0.80,  7.50],
    ('Cardboard', 'Packaging')            : [0.70,  8.50]})

**Step 3: Set up the Linear Programming Model**

In [15]:
# Create a Gurobi model
m = gp.Model("Paper_recycling")

# Define the decision variables
x = m.addVars(material_product_pairs,name='Material_Product_pairs')

# Define the objective function
m.setObjective(sum((purchase_cost[i] + processing_cost[i,j]) * x[i,j] for i,j in material_product_pairs), sense=GRB.MINIMIZE)

# Define the constraints
m.addConstrs(sum(x[i,j] for j in products if (i,j) in material_product_pairs) <= availability[i] for i in materials)

m.addConstrs(sum(recycling_yield[i,j] * x[i,j] for i in materials if (i,j) in material_product_pairs) >= product_demand[j] for j in products)




{'Newsprint': <gurobi.Constr *Awaiting Model Update*>,
 'Packaging': <gurobi.Constr *Awaiting Model Update*>,
 'Print stock': <gurobi.Constr *Awaiting Model Update*>}

**Step 4: Solve the Model and Display Output**

In [16]:
# Solve the model
m.optimize()

# Display the results
print(f"Minimum Cost: ${m.objVal:.2f}")

for i,j in material_product_pairs:
    print(f"{x[i,j].x:.2f} tons of {i} is processed to produce {j}.")


Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (linux64)

CPU model: AMD EPYC 7B12, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 7 rows, 10 columns and 20 nonzeros
Model fingerprint: 0xb1e3ed65
Coefficient statistics:
  Matrix range     [7e-01, 1e+00]
  Objective range  [2e+01, 3e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+02, 6e+02]
Presolve time: 0.01s
Presolved: 7 rows, 10 columns, 20 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   1.400000e+03   0.000000e+00      0s
       7    4.4067737e+04   0.000000e+00   0.000000e+00      0s

Solved in 7 iterations and 0.02 seconds (0.00 work units)
Optimal objective  4.406773709e+04
Minimum Cost: $44067.74
588.24 tons of Newspaper is processed to produce Newsprint.
11.76 tons of Newspaper is processed to produce Packaging.
0.00 tons of Mixed paper is processed to produce Newsprint.
71.4